In [ ]:
!pip install wandb

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import wandb

In [3]:
wandb.init(project="17-Flowers-by-Torch")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.resnet50(pretrained=True)

## Transfer Learning

In [5]:
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 17)

This freezes layers 1-6 in the total 10 layers of Resnet50

In [6]:
ct = 0
for child in model.children():
    ct += 1
    if ct < 7:
        for param in child.parameters():
            param.requires_grad = False

In [7]:
model = model.to(device)

In [8]:
config = wandb.config
config.learning_rate = 0.01
config.batch_size = 32
config.epochs = 15

Dataset

In [9]:
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Datasets/17flowers', transform=transform)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size, shuffle=True, num_workers=2)

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
loss_function = torch.nn.CrossEntropyLoss()

In [11]:
def calc_acc(preds, labels):
    preds_max = torch.argmax(preds, 1)
    acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
    return acc

## Train

In [12]:
wandb.watch(model)

for epoch in range(config.epochs):
    train_loss = 0.0
    train_acc = 0.0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # 1- forwarding
        preds = model(images)
        # 2- backwarding 
        loss = loss_function(preds, labels)
        loss.backward()
        # 3- Update
        optimizer.step()

        train_loss += loss
        train_acc += calc_acc(preds, labels)
    
    total_loss = train_loss / len(train_data_loader)
    total_acc = train_acc / len(train_data_loader)

    if epoch % 2 == 0:
        wandb.log({"loss": total_loss})
        wandb.log({"acc": total_acc})

    print(f"Epoch: {epoch}, Loss: {total_loss}, Acc: {total_acc}")

Epoch: 0, Loss: 3.4751694202423096, Acc: 0.0806686046511628
Epoch: 1, Loss: 2.451564073562622, Acc: 0.19912790697674418
Epoch: 2, Loss: 1.7528003454208374, Acc: 0.407703488372093
Epoch: 3, Loss: 1.1625523567199707, Acc: 0.6162790697674418
Epoch: 4, Loss: 0.7201101779937744, Acc: 0.7638081395348837
Epoch: 5, Loss: 0.5774242281913757, Acc: 0.8168604651162791
Epoch: 6, Loss: 0.4540393352508545, Acc: 0.8546511627906976
Epoch: 7, Loss: 0.3330390751361847, Acc: 0.8938953488372093
Epoch: 8, Loss: 0.3088412582874298, Acc: 0.904796511627907
Epoch: 9, Loss: 0.32221150398254395, Acc: 0.8938953488372093
Epoch: 10, Loss: 0.21559959650039673, Acc: 0.9331395348837209
Epoch: 11, Loss: 0.13378223776817322, Acc: 0.9505813953488372
Epoch: 12, Loss: 0.18655388057231903, Acc: 0.9396802325581395
Epoch: 13, Loss: 0.16620847582817078, Acc: 0.9505813953488372
Epoch: 14, Loss: 0.15996012091636658, Acc: 0.9563953488372093
